# KoBERT finetuning

In [ ]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments

In [ ]:
# 모델 및 토크나이저 로드
model_name = "skt/kobert-base-v1"  # 또는 다른 KoBERT 모델
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # num_labels는 분류할 클래스 수에 따라 변경

In [ ]:
# NSMC 영화 리뷰 데이터셋 준비
dataset = load_dataset("e9t/nsmc", trust_remote_code=True)

def tokenize_function(example):
    return tokenizer(example["document"])

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
# 데이터 길이를 똑같이 맞추기
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="./results",          # 결과 저장 디렉토리
    learning_rate=5e-5,
    per_device_train_batch_size=64,  # 훈련 배치 크기
    per_device_eval_batch_size=64,   # 평가 배치 크기
    num_train_epochs=5,              # 훈련 에폭 수
    weight_decay=0.01,               # 가중치 감소
    eval_strategy="epoch",     # 에폭마다 평가 수행
    save_strategy="epoch",          # 에폭마다 모델 저장
    load_best_model_at_end=True,     # 훈련 종료 시 가장 좋은 모델 로드
    metric_for_best_model="accuracy", # 가장 좋은 모델을 선택하기 위한 메트릭
    push_to_hub=False,               # Hugging Face Hub에 모델 업로드 여부
    max_steps=10,
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": np.mean(predictions == labels)}

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 훈련 시작
trainer.train()

In [ ]:
# 평가 진행
trainer.evaluate()